Make sure to install beautifulsoup4!

In [6]:
#!pip install beautifulsoup4
#!pip install lxml

You should consider upgrading via the 'c:\python3.9.2\python.exe -m pip install --upgrade pip' command.


In [65]:
import pandas as pd
import re
from pathlib import Path
import requests
from datetime import datetime
from bs4 import BeautifulSoup

In [70]:
def get_soup(url):
    html_content = requests.get(url).text
    soup = BeautifulSoup(html_content, "lxml")
    print(soup.prettify()) # print the parsed data of html
    return soup

In [72]:
def scrap_icici_reports(icici_url, csvfile, reportsdir):
    """
    Function to scrap icici reports page. Can be called multiple times,
    without worrying about redownloads. Only new reports, if avaialble, are downloaded.
    Delete saved csvfile and content of reportsdir to redownload reports.
    
    icici_url : Link to reports page.
    csvfile : Filename ending with '.csv', to save details and highlights of reports.
    reportsdir : Name of directory to save reports. Must already exist. 
    """
    
    page = get_soup(icici_url)
    text_entries = []
    link_entries = {}
    lastline = None

    try:
        with open(csvfile, "r") as f:
            for line in f: 
                pass
            lastline = line if line else None  #this is the last line of the file
    except FileNotFoundError:
        lastline = None


    for irow, row in enumerate(page.body.find_all('tbody')[2].find_all('tr')):
        if irow == 0:
            continue
        text = []
        links = []
        for i, cell in enumerate(row.find_all('td')):
            #text.append(cell.text.strip().replace("\t", "").replace("\r", "").replace("\n",""))
            text.append(re.sub('\s+',' ', cell.text).replace(',', ''))
            if len(cell.find_all('a')) > 0:
                link = cell.find_all('a')[0]
                links.append(link.get('href'))

        newline = ','.join(text)+'\n'
        if newline == lastline:
            # print(newline, lastline, newline==lastline)
            break
        else:
            text_entries.append(newline)
            if links:
                link_entries[(text[0] + text[1]).replace(',','')] = links[-1]

    if text_entries:
        with open(csvfile, 'a') as f:
            for entry in text_entries[::-1]:
                f.write(entry.encode('ascii', 'ignore').decode('ascii'))

    for fn, url in link_entries.items():
        filename = reportsdir+'/'+fn.strip()+'.pdf'
        response = requests.get(url)
        with open(filename, 'wb') as f:
            f.write(response.content)

        with open('log.txt', 'a') as logfile:
            logfile.write("\n{} : {} scrapped.".format(datetime.now(), filename))

In [73]:
icici_url = "https://www.icicidirect.com/mailimages/Co_reports.htm"
csvfile = 'reports_data.csv'
reportsdir = 'reports'

scrap_icici_reports(icici_url, csvfile, reportsdir)
print('Execution finished.')

<!DOCTYPE HTML PUBLIC "-//W3C//DTD HTML 4.0 Transitional//EN">
<!-- saved from url=(0051)http://172.16.11.31/mailimages/detailed_reports.htm -->
<!-- saved from url=(0041)file://Z:\Production\detailed_reports.htm -->
<!-- saved from url=(0041)file://Z:\Production\detailed_reports.htm -->
<html>
 <head>
  <title>
   Date
  </title>
  <meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
  <meta content="MSHTML 6.00.6000.21357" name="GENERATOR"/>
  <meta content="FrontPage.Editor.Document" name="ProgId"/>
  <style type="text/css">
   .auto-style1 {
	font-family: Arial;
	font-size: x-small;
}
.auto-style2 {
	font-weight: normal;
}
.auto-style3 {
	font-family: Arial;
	font-size: x-small;
	color: #0000FF;
}
.auto-style4 {
	text-decoration: underline;
}
.auto-style5 {
	font-size: x-small;
}
.auto-style6 {
	color: #0000FF;
}
.auto-style7 {
	font-size: x-small;
	color: #0000FF;
}
.auto-style8 {
	color: #FFFFFF;
	font-family: "Zurich BT";
	font-size: x-small;
}
.auto-style9 {
	fo

KeyboardInterrupt: 